# Ex9 아이유팬이 좋아할 만한 다른 아티스트 찾기

## Movielens 영화 추천 실습

### 유저 아이디, 영화 아이디, 별점, 시간이 담긴 rating.dat파일을 읽어옵니다.

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 별점이 3점이상일 때만 선호하는것으로 여기므로 3이상 데이터만 추출합니다.

In [2]:
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


###  별점을 counts로 이름을 바꿉니다.

In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings['counts']
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 필요없는 열(timestamp)는 삭제합니다.

In [4]:
ratings=ratings.drop(['timestamp'],axis=1)
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


### 영화의 이름 장르가 담긴 movies.dat파일을 읽어옵니다.

In [5]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 필요없는 열을 삭제합니다.(장르)

In [6]:
movies=movies.drop(['genre'],axis=1)
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


### 1. ratings에 있는 유니크한 영화 개수 
### 2. ratings에 있는 유니크한 사용자 개수
### 3. 가장 인기있는 영화 30개 (인기순) 을 출력해 봤습니다.

In [7]:
num_movie = ratings['movie_id'].nunique()
num_user = ratings['user_id'].nunique()
print(f"num movie: {num_movie}")
print(f"num user: {num_user}")


num movie: 3628
num user: 6039


In [8]:
print(len(ratings))
print(len(movies))

836478
3883


### 인기순 영화 제목을 뽑기위해 ratings와 movies를 movie_id 기준으로 merge 했습니다. 

In [9]:
data=pd.merge(ratings,movies)
data.head()

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975)
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975)
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975)
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975)


### user_id기준으로 groupby를 하여 가장 많이본 순서대로 이름을 출력했습니다.

In [10]:
movie_count = data.groupby('title')['user_id'].count()
data2=movie_count.sort_values(ascending=False).head(30)
data2

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 제가 좋아하는 영화 5개를 데이터의 마지막부분에 삽입했습니다.

In [11]:
my_favorite = ['Star Wars: Episode IV - A New Hope (1977)','Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode VI - Return of the Jedi (1983)' ,'Men in Black (1997)','E.T. the Extra-Terrestrial (1982)']

my_playlist = pd.DataFrame({'user_id': ['ykb']*5, 'title': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':['ykb']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.


,user_id,movie_id,counts,title
836473,5851,3607.0,5,One Little Indian (1973)
836474,5854,3026.0,4,Slaughterhouse (1987)
836475,5854,690.0,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,2909.0,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,1360.0,5,Identification of a Woman (Identificazione di ...
0,ykb,NaN,5,Star Wars: Episode IV - A New Hope (1977)
1,ykb,NaN,5,Star Wars: Episode V - The Empire Strikes Back...
2,ykb,NaN,5,Star Wars: Episode VI - Return of the Jedi (1983)
3,ykb,NaN,5,Men in Black (1997)
4,ykb,NaN,5,E.T. the Extra-Terrestrial (1982)


### movie_id는 이제 필요없어지므로 삭제합니다.

In [12]:
data=data.drop(['movie_id'],axis=1)
data.tail(10)

,user_id,counts,title
836473,5851,5,One Little Indian (1973)
836474,5854,4,Slaughterhouse (1987)
836475,5854,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,5,Identification of a Woman (Identificazione di ...
0,ykb,5,Star Wars: Episode IV - A New Hope (1977)
1,ykb,5,Star Wars: Episode V - The Empire Strikes Back...
2,ykb,5,Star Wars: Episode VI - Return of the Jedi (1983)
3,ykb,5,Men in Black (1997)
4,ykb,5,E.T. the Extra-Terrestrial (1982)


### 유저 ID와 영화제목에 대한 인덱싱 값을 만듭니다.

In [13]:
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

print(user_to_idx['ykb']) 


6039


### 유저아이디와 타이틀에대하여 인덱싱값을 데이터에다가 넣어줬습니다.

In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data



user_id column indexing OK!!
movie column indexing OK!!


,user_id,counts,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,44
1,6039,5,117
2,6039,5,64
3,6039,5,175


### scr 메트릭스를 가공한 데이터 기반으로 만들었습니다.

In [15]:
from scipy.sparse import csr_matrix

num_movie = data['title'].nunique()
num_user = data['user_id'].nunique()
csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 기본 세팅값을 설정해줬습니다.

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

### 훈련할때의 파라미터값 성정했습니다.

In [17]:
als_model = AlternatingLeastSquares(factors=1500, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 제가 좋아하는 영화 중 하나인 맨인블랙과 저의 유저아이디 ykb에 대한 벡터를 뽑았습니다.

In [20]:
ykb, MeninBlack = user_to_idx['ykb'], movie_to_idx['Men in Black (1997)']
ykb_vector, MeninBlack_vector = als_model.user_factors[ykb], als_model.item_factors[MeninBlack]

print('슝=3')

슝=3


In [21]:
ykb_vector

array([ 2.19790578e-01, -1.09886885e+00, -7.16196775e-01,  3.54425460e-01,
        2.80894518e-01,  5.37232041e-01,  2.54187524e-01,  9.18065608e-02,
        5.55480659e-01, -2.95272022e-01,  5.73143773e-02,  1.01621956e-01,
       -6.25047028e-01, -1.64370507e-01, -2.09388752e-02, -2.48384550e-01,
       -5.38793325e-01,  4.22819525e-01, -3.53986800e-01, -9.35392361e-03,
       -2.26437405e-01, -1.29937038e-01,  7.75648952e-01, -2.81162471e-01,
        3.72987747e-01,  5.73588669e-01,  4.61037159e-02, -1.96407825e-01,
       -4.95271564e-01,  5.47562063e-01,  5.24464212e-02, -6.08492017e-01,
        1.33233652e-01,  1.72935203e-01, -7.10373819e-01,  8.79221737e-01,
        3.24885398e-01, -1.66777328e-01,  6.56710505e-01,  4.59131569e-01,
        2.79026657e-01, -6.58334270e-02, -1.68131053e-01,  1.47500724e-01,
        3.46129149e-01,  4.40572679e-01,  2.21618712e-01,  7.14963824e-02,
        3.60649675e-01, -5.81205845e-01, -4.27757323e-01,  4.58763570e-01,
        1.50751337e-01, -

### 내적을하니 0.77가 나왔습니다.

In [22]:
np.dot(ykb_vector, MeninBlack_vector)

0.7711379

###  제가 좋아하는 영화리스트에 없는 영화중 아무거나를 넣어 제 아이디와 내적을한 결과 매우 낮은 값이 나왔습니다.

In [23]:
Indian = movie_to_idx['One Little Indian (1973)']
Indian_vector = als_model.item_factors[Indian]
np.dot(ykb_vector, Indian_vector)

0.0031649247

### 스타워즈와 유사한 영화 리스트를 뽑았습니다.

In [24]:
favorite_movie = 'Star Wars: Episode IV - A New Hope (1977)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Reckless (1995)',
 'Desert Blue (1999)',
 'Bushwhacked (1995)',
 'Zachariah (1971)',
 'Sunset Strip (2000)',
 'Clean Slate (Coup de Torchon) (1981)',
 'Closer You Get, The (2000)',
 'Boys (1996)',
 'Hav Plenty (1997)',
 'Deadtime Stories (1987)',
 'Just the Ticket (1999)',
 'Gold Diggers: The Secret of Bear Mountain (1995)',
 'Match, The (1999)']

### 제가 좋아할만한 영화 리스트를 받았습니다.

In [25]:
user = user_to_idx['ykb']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
[idx_to_movie[i[0]] for i in movie_recommended]

['Jurassic Park (1993)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Raiders of the Lost Ark (1981)',
 'Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Forrest Gump (1994)',
 'Galaxy Quest (1999)',
 'Face/Off (1997)',
 'Wizard of Oz, The (1939)',
 'Superman (1978)',
 'Close Encounters of the Third Kind (1977)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 '2001: A Space Odyssey (1968)',
 'Babe (1995)',
 'Remember the Titans (2000)',
 'Back to the Future (1985)',
 'Willy Wonka and the Chocolate Factory (1971)']

# 회고

* 제가 훈련시킨 모델을 기반으로 스타워즈(SF,액션,판타지)영화와 비슷한 영화를 뽑아봤습니다. 그결과 스타워즈의 다른 시리즈물이 나오고 터미네이터, 백투더퓨처, 매트릭스 등 대부분의 추천영화가 SF, 액션 판타지 영화가 나와 만족스런 결과가 나왔습니다.

* 제가 좋아하는영화인 SF,판타지,액션영화들을 목록에 넣고 모델을 훈련시킨다음 제가 좋아할많나 영화 리스트를 추천받아 봤습니다. 그 결과 쥬라기공원, 터미네이터, 매트릭스 등 SF, 판타지, 액션류의 영화를 잘 추천받았습니다.

* 사용자와 아이템 벡터 내적수치를 올리기 위해 사용자와 아이템의 차원수를 계속 올려서 0.77까지 올려봤습니다. 그 결과 제가 의도했던 sf, 판타지, 액션 영화들만 추천하는 것이 아니라 다른 종류의 영화가 추천을 했습니다. 왜그런가 생각해보니 사용자가 sf물을 좋아한다고 sf영화만 보지않을 수 있고 연기자, 감독 등 여러가지 요소로 영화 선택을 할 수 있다고 생각합니다. 그래서 영화 추천이 좀더 다양한 장르로 추천이 됐다고 생각합니다.   